# State Farm (no data augmentation)

## Steps:

1. load train and validation data
2. load VGG model
3. remove final layer and precompute
4. replace final layer
5. train
6. test

## 1. Load data

In [17]:
floydhub = False
sample = True

In [31]:
if floydhub:
    data_path = "/input/"
    bottleneck_path = "/output/bottlenecks/"
    results_path = "/output/results/"
else:
    data_path = "./data/"
    if sample:
        data_path += "sample/"
    bottleneck_path = "./bottlenecks/"
    results_path = "./results/"
%mkdir $bottleneck_path
%mkdir $results_path

mkdir: cannot create directory './data/sample/': File exists


In [35]:
batch_size = 8

In [22]:
from utils import get_data, get_classes
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()

train_data = get_data(data_path + "train")
valid_data = get_data(data_path + "valid")

Found 1817 images belonging to 10 classes.
Found 1011 images belonging to 10 classes.
Found 1817 images belonging to 10 classes.
Found 1011 images belonging to 10 classes.


OSError: [Errno 2] No such file or directory: './data/sample/test'

In [23]:
val_classes, train_classes, val_labels, train_labels, _,_,_ = get_classes(data_path)

Found 1817 images belonging to 10 classes.
Found 1011 images belonging to 10 classes.
Found 1011 images belonging to 10 classes.


## 2. Load VGG

In [40]:
from vgg16 import Vgg16
vgg = Vgg16()

vgg.model.pop()
vgg.model.pop()
vgg.model.pop()
vgg.compile()

/opt/conda/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [41]:
train_bottlenecks = vgg.model.predict(train_data)

In [42]:
valid_bottlenecks = vgg.model.predict(valid_data)

In [43]:
from utils import save_array
save_array(bottleneck_path+"train_bottlenecks.bc", train_bottlenecks)
save_array(bottleneck_path+"valid_bottlenecks.bc", valid_bottlenecks)

## 4. Train classification network

In [50]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization

for layer in vgg.model.layers:
    layer.trainable = False
fc_model = Sequential([
    BatchNormalization(input_shape=train_bottlenecks.shape[1:]),
    Dropout(0.7),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(10, activation="softmax")
])

fc_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fc_model.summary()

In [51]:
fc_model.fit(train_bottlenecks, train_labels, nb_epoch=10, validation_data=(valid_bottlenecks, val_labels), batch_size=batch_size, verbose=2)

Train on 1817 samples, validate on 1011 samples
Epoch 1/10
39s - loss: 1.0903 - acc: 0.7700 - val_loss: 5.8604 - val_acc: 0.3640
Epoch 2/10
38s - loss: 0.5615 - acc: 0.8899 - val_loss: 8.8555 - val_acc: 0.2384
Epoch 3/10
38s - loss: 0.4586 - acc: 0.9048 - val_loss: 10.5621 - val_acc: 0.2463
Epoch 4/10
38s - loss: 0.4519 - acc: 0.9119 - val_loss: 11.6690 - val_acc: 0.1820
Epoch 5/10
38s - loss: 0.3413 - acc: 0.9329 - val_loss: 10.7049 - val_acc: 0.2265
Epoch 6/10
38s - loss: 0.3797 - acc: 0.9285 - val_loss: 10.5933 - val_acc: 0.2878
Epoch 7/10


KeyboardInterrupt: 